# 1. Load Data

A familiar dataset which I've played with and have parsed, needs to be changed to the other datasets which the paper uses

In [1]:
from sklearn import datasets
import numpy

x_sparse, y = datasets.load_svmlight_file('diabetes')
x = x_sparse.todense()

print('Shape of x: ' + str(x.shape))
print('Shape of y: ' + str(y.shape))

Shape of x: (768, 8)
Shape of y: (768,)


In [2]:
# partition the data to training and test sets
n = x.shape[0]
n_train = 640
n_test = n - n_train

rand_indices = numpy.random.permutation(n)
train_indices = rand_indices[0:n_train]
test_indices = rand_indices[n_train:n]

x_train = x[train_indices, :]
x_test = x[test_indices, :]
y_train = y[train_indices].reshape(n_train, 1)
y_test = y[test_indices].reshape(n_test, 1)

print('Shape of x_train: ' + str(x_train.shape))
print('Shape of x_test: ' + str(x_test.shape))
print('Shape of y_train: ' + str(y_train.shape))
print('Shape of y_test: ' + str(y_test.shape))

Shape of x_train: (640, 8)
Shape of x_test: (128, 8)
Shape of y_train: (640, 1)
Shape of y_test: (128, 1)


In [3]:
# Standardization
import numpy

# calculate mu and sig using the training set
d = x_train.shape[1]
mu = numpy.mean(x_train, axis=0).reshape(1, d)
sig = numpy.std(x_train, axis=0).reshape(1, d)

# transform the training features
x_train = (x_train - mu) / (sig + 1E-6)

# transform the test features
x_test = (x_test - mu) / (sig + 1E-6)

print('test mean = ')
print(numpy.mean(x_test, axis=0))

print('test std = ')
print(numpy.std(x_test, axis=0))

test mean = 
[[-0.00323299  0.04022588  0.0292436   0.05085141  0.03199379  0.03404022
   0.17395318  0.05570114]]
test std = 
[[0.97180563 0.93688322 1.08138887 1.00845326 1.19543904 0.90443632
  0.92388706 1.05479545]]


In [4]:
n_train, d = x_train.shape
x_train = numpy.concatenate((x_train, numpy.ones((n_train, 1))), axis=1)

n_test, d = x_test.shape
x_test = numpy.concatenate((x_test, numpy.ones((n_test, 1))), axis=1)

print('Shape of x_train: ' + str(x_train.shape))
print('Shape of x_test: ' + str(x_test.shape))

Shape of x_train: (640, 9)
Shape of x_test: (128, 9)


# 2. Binary Classification

## Logistic Loss Objective Value

Objective Function:
$ q (w; x, y) = \log \Big( 1 + \exp \big( - y_i x_i^T w \big) \Big)$ 

l2 regularizer addition:
$\frac{\lambda}{2} \| w \|_2^2 $.

l1 regularizer addition: 
$\lambda| w \|^1 $

In [5]:
# y: scalar 
# x: 1 by d vector 
# w: d by 1 vector
# lam: scalar 
# l1 and l2: boolean
def logistic_loss(y,x,w,l1=False,l2=False,lam=0):
    exponent = float(numpy.exp(-y * numpy.dot(x,w))) # scalar
    objective = numpy.log(1 + exponent) 
    if l2:
        objective += lam/2 * numpy.sum(numpy.multiply(w,w))
    if l1:
        objective += numpy.sum(numpy.abs(w)) 
    return objective

In [6]:
# gradient of logisitic loss
# y: scalar 
# x: 1 by d vector 
# w: d by 1 vector
# lam: scalar 
# l1 and l2: boolean
def logistic_gradient(y,x,w,l1=False,l2=False, lam=0):
    d, _ = w.shape
    exponent = float(numpy.exp(y * numpy.dot(x,w))) # scalar
    derivative = (-y * x) / (1 + exponent) 
    derivative = derivative.reshape((d,1))# d by 1
    if l2:
        derivative += lam * w
    if l1:
        derivative += numpy.multiply(numpy.sign(w),w)
    return derivative

# Regression

## Linear Regression

TODO: write equations

In [7]:
# y: scalar 
# x: 1 by d vector 
# w: d by 1 vector
# lam: scalar 
# l1 and l2: boolean
def linear_loss(y,x,w,l2=False,l1=False,lam=0):
    loss = numpy.dot(x,w) - y
    loss = numpy.multiply(loss,loss)
    if l2:
        loss += lam * numpy.multiply(w,w)
    if l1:
        loss += lam * numpy.sum(numpy.abs(w))
    return loss

In [8]:
# y: scalar 
# x: 1 by d vector 
# w: d by 1 vector
# lam: scalar 
# l1 and l2: boolean
def linear_gradient(y,x,w,l2=False,l1=False,lam=0):
    d, _ = w.shape
    scalar = numpy.dot(w,x) - y
    vector = scalar * x 
    vector.reshape((d,1))
    if l2:
        vector += lam * w
    if l1:
        vector += numpy.multiply(numpy.sign(w),w)
    return vector

## SAGA Algorithm

In [9]:
# X: training set, n by d 
# y: training labels, n by 1 
# step_size, can't go wrong with 1/3L
# lam: scalar
# max_epochs = scalar
# proximal: not always used default should be lambda x
# obj_func: R^d -> R^1
# grad_func: R^d -> R^d
# l1 and l2 = Booleans
def saga(X,y,step_size,max_epochs,proximal,obj_func,grad_func,l1=False,l2=False,lam=0):
    # average obj values per epoch
    obj_vals = []
    n, d = X.shape  
    w = numpy.zeros((d,1)) # d by 1 weight vector
    # initialize table with derivative w/weight 0
    derivatives = numpy.zeros((n,d))
    for i in range(d):
        derivatives[i,:] = grad_func(y[i],X[i,:],w,l1,l2,lam).reshape(9)
        
    for epoch in range(max_epochs):
        # shuffle data points for an epoch
        permutation = numpy.random.permutation(n)
        X_shuffled = X[permutation,:]
        y_shuffled = y[permutation,:]
        obj_epoch = 0
        for i in range(n):
            # target data point and label
            xi = X_shuffled[i,:]
            yi = y_shuffled[i]

            updated_deriv = grad_func(yi,xi,w,l1=l1,l2=l2,lam=lam) # d by 1
            previous_deriv = derivatives[permutation[i],:].reshape((d,1)) # d by 1
            derivatives[permutation[i],:] = updated_deriv.reshape(d)
            table_avg = numpy.mean(derivatives,axis=0).reshape((d,1))
            update = updated_deriv - previous_deriv + table_avg
            w = w - (step_size * update)
            # apply proximal operator
            w = proximal(w)
            obj_iter = obj_func(y_shuffled[i],X_shuffled[i,:],w,l1=l1,l2=l2,lam=lam)
            obj_epoch += obj_iter

        obj_epoch /= n
        obj_vals.append(obj_epoch)
        print("Obj val at epoch " + str(epoch) + ' is ' + str(obj_epoch))

In [ ]:
proximal = lambda x: x
_,eigs,_ = numpy.linalg.svd(x_train * x_train.T)
n,d = numpy.shape(x_train)
alpha = 1E-6
step_size = 1 / (n * alpha + 1/4 * eigs[0]) 
saga(x_train,y_train,step_size,100,proximal,logistic_loss,logistic_gradient,l2=True,lam=alpha)

Obj val at epoch 0 is 0.6588002630514876
Obj val at epoch 1 is 0.5515855680421203
Obj val at epoch 2 is 0.5104804259070631
Obj val at epoch 3 is 0.49584571218481754
Obj val at epoch 4 is 0.4885791791052088
Obj val at epoch 5 is 0.48431571830607245
Obj val at epoch 6 is 0.4818573108614399
Obj val at epoch 7 is 0.4801292759831207
Obj val at epoch 8 is 0.4792396018693451
Obj val at epoch 9 is 0.47820711633892243
Obj val at epoch 10 is 0.47778266331929514
Obj val at epoch 11 is 0.47763266457157705
Obj val at epoch 12 is 0.4773093432078349
Obj val at epoch 13 is 0.4772009244614339
Obj val at epoch 14 is 0.4770749871833301
Obj val at epoch 15 is 0.477013897631785
Obj val at epoch 16 is 0.4769733361655414
Obj val at epoch 17 is 0.47687696721618406
Obj val at epoch 18 is 0.47686376320372653
Obj val at epoch 19 is 0.4768044854767887
Obj val at epoch 20 is 0.4767733835220655
Obj val at epoch 21 is 0.47677463436519807
Obj val at epoch 22 is 0.4767707909431412
Obj val at epoch 23 is 0.476765905269